In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
for_hire_vehicle_data = pd.read_csv('/Users/disastershubz/Documents/PROJECT/For-Hire Vehicle Trip Records .csv')
green_taxi_data = pd.read_csv('/Users/disastershubz/Documents/PROJECT/green_taxi_data.csv')
citibike_data = pd.read_csv('/Users/disastershubz/Documents/PROJECT/citibike_data.csv')
yellow_taxi_data = pd.read_csv('/Users/disastershubz/Documents/PROJECT/yellow_taxi_data.csv')

In [5]:
taxi_zones = pd.read_csv('/Users/disastershubz/Documents/PROJECT/taxi_zones.csv')

In [6]:
citibike_stations = pd.read_csv('/Users/disastershubz/Documents/PROJECT/citibike_stations.csv')

In [7]:
green_taxi_data.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2024-07-01 00:44:19,2024-07-01 01:14:35,N,1.0,80,100,2.0,8.20,38.0,1.0,0.5,2.00,0.0,0.0,1.0,45.25,1.0,1.0,2.75
1,2,2024-07-01 00:26:06,2024-07-01 00:51:58,N,1.0,66,89,1.0,7.87,36.6,1.0,0.5,7.82,0.0,0.0,1.0,46.92,1.0,1.0,0.00
2,2,2024-07-01 00:36:44,2024-07-01 00:46:07,N,1.0,129,260,1.0,0.96,8.6,1.0,0.5,0.00,0.0,0.0,1.0,11.10,2.0,1.0,0.00
3,2,2024-07-01 00:35:40,2024-07-01 00:47:47,N,1.0,42,120,1.0,5.73,24.0,1.0,0.5,0.00,0.0,0.0,1.0,26.50,1.0,1.0,0.00
4,2,2024-07-01 00:36:53,2024-07-01 00:56:58,N,1.0,152,185,1.0,8.62,35.2,1.0,0.5,11.31,0.0,0.0,1.0,49.01,1.0,1.0,0.00


In [8]:
green_taxi_data.shape

(46827, 20)

In [9]:
yellow_taxi_data.shape

(2729770, 19)

In [10]:
citibike_data.shape

(112443, 13)

In [11]:
citibike_stations.shape

(812, 13)

In [12]:
taxi_zones.shape

(263, 7)

In [13]:
yellow_taxi_data.shape

(2729770, 19)

In [23]:
# Group by pickup and drop-off zones to get counts
yellow_pickup_counts = yellow_taxi_data.groupby('PULocationID').size().rename('yellow_pickup_count')
yellow_dropoff_counts = yellow_taxi_data.groupby('DOLocationID').size().rename('yellow_dropoff_count')
green_pickup_counts = green_taxi_data.groupby('PULocationID').size().rename('green_pickup_count')
green_dropoff_counts = green_taxi_data.groupby('DOLocationID').size().rename('green_dropoff_count')

# Merge these counts into one DataFrame
taxi_density = pd.DataFrame({
    'yellow_pickup_count': yellow_pickup_counts,
    'yellow_dropoff_count': yellow_dropoff_counts,
    'green_pickup_count': green_pickup_counts,
    'green_dropoff_count': green_dropoff_counts
}).fillna(0)  # Fill NaNs with 0s in case some zones have no counts


#  Location-Based Recommendations: For high-density pickup/drop-off zones, can Citibike stations help relieve taxi congestion?


In [25]:
# Ensure all columns are converted to numeric types where possible
taxi_density = taxi_density.apply(lambda x: pd.to_numeric(x, errors='ignore') if x.dtype.name == 'category' else x)

# Check if any columns are still non-numeric and convert if needed
for column in taxi_density.columns:
    if taxi_density[column].dtype.name == 'category' or pd.api.types.is_categorical_dtype(taxi_density[column]):
        taxi_density[column] = taxi_density[column].astype('float64')  # Convert to float to handle NaNs

# Now we can safely fill NaN values with 0
taxi_density = taxi_density.fillna(0)

# Calculate total activity per zone
taxi_density['total_taxi_activity'] = taxi_density[['yellow_pickup_count', 'yellow_dropoff_count', 
                                                    'green_pickup_count', 'green_dropoff_count']].sum(axis=1)

# Check the result
print(taxi_density.head())


   yellow_pickup_count  yellow_dropoff_count  green_pickup_count  \
1                102.0                7851.0                 1.0   
2                  5.0                   5.0                 0.0   
3                111.0                 281.0                 0.0   
4               2479.0                9181.0                 2.0   
5                  0.0                  17.0                 0.0   

   green_dropoff_count  total_taxi_activity  
1                 23.0               7977.0  
2                  0.0                 10.0  
3                  3.0                395.0  
4                 34.0              11696.0  
5                  0.0                 17.0  


# Filter for High-Density Zones

In [30]:
# Define a high-density threshold
high_density_threshold = 5000

# Filter taxi density DataFrame to get high-density zones
high_density_zones = taxi_density[taxi_density['total_taxi_activity'] > high_density_threshold]
print("High-Density Taxi Zones:")
print(high_density_zones)


High-Density Taxi Zones:
     yellow_pickup_count  yellow_dropoff_count  green_pickup_count  \
1                  102.0                7851.0                 1.0   
4                 2479.0                9181.0                 2.0   
7                 1322.0                6472.0               682.0   
13               17210.0               20371.0                 2.0   
24                5583.0                7753.0               302.0   
..                   ...                   ...                 ...   
261              16245.0               14558.0                 2.0   
262              30816.0               37355.0                 4.0   
263              46267.0               52895.0                44.0   
264               8464.0               10735.0                40.0   
265                766.0               16038.0                24.0   

     green_dropoff_count  total_taxi_activity  
1                   23.0               7977.0  
4                   34.0              

# Step 2: Find Nearby CitiBike Stations for High-Density Zones

In [35]:
# Add LocationID column to taxi_density based on `taxi_zones`
taxi_density = taxi_density.merge(taxi_zones[['LocationID', 'zone', 'borough']], 
                                  left_index=True, right_on='LocationID', how='left')

# Filter for high-density zones
high_density_threshold = 5000
high_density_zones = taxi_density[taxi_density['total_taxi_activity'] > high_density_threshold]

# Now, merge high-density zones with CitiBike stations based on location proximity
# Assuming citibike_stations has 'station_id', 'name', 'capacity', 'lat', 'lon' columns

# Join high_density_zones with citibike_stations
nearby_stations = high_density_zones.merge(citibike_stations, left_on='LocationID', right_on='station_id', how='left')

print("High-Density Taxi Zones with Nearby CitiBike Stations:")
print(nearby_stations[['zone', 'borough', 'total_taxi_activity', 'name', 'capacity', 'lat', 'lon']])


High-Density Taxi Zones with Nearby CitiBike Stations:
                  zone    borough  total_taxi_activity  \
0       Newark Airport        EWR               7977.0   
1        Alphabet City  Manhattan              11696.0   
2              Astoria     Queens               8959.0   
3    Battery Park City  Manhattan              37638.0   
4         Bloomingdale  Manhattan              14098.0   
..                 ...        ...                  ...   
76  World Trade Center  Manhattan              30846.0   
77      Yorkville East  Manhattan              68993.0   
78      Yorkville West  Manhattan             100598.0   
79                 NaN        NaN              19397.0   
80                 NaN        NaN              17027.0   

                        name  capacity        lat        lon  
0                        NaN       NaN        NaN        NaN  
1                        NaN       NaN        NaN        NaN  
2                        NaN       NaN        NaN        Na

In [37]:
nearby_stations.head()

,yellow_pickup_count,yellow_dropoff_count,green_pickup_count,green_dropoff_count,total_taxi_activity,LocationID,zone,borough,capacity,eightd_has_key_dispenser,...,external_id,has_kiosk,lat,lon,name,region_id,rental_methods,rental_url,short_name,station_id
0,102.0,7851.0,1.0,23.0,7977.0,1,Newark Airport,EWR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2479.0,9181.0,2.0,34.0,11696.0,4,Alphabet City,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1322.0,6472.0,682.0,483.0,8959.0,7,Astoria,Queens,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17210.0,20371.0,2.0,55.0,37638.0,13,Battery Park City,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5583.0,7753.0,302.0,460.0,14098.0,24,Bloomingdale,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
!pip install folium


In [43]:
import folium
from folium.plugins import MarkerCluster

# Initialize the map centered around a location in NYC
nyc_map = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

nearby_stations = nearby_stations.dropna(subset=['lat', 'lon'])

# Plot high-density taxi zones on the map
taxi_cluster = MarkerCluster(name="High-Density Taxi Zones").add_to(nyc_map)
for _, row in nearby_stations.iterrows():
    # Add high-density taxi zones as larger markers, color-coded by activity
    folium.CircleMarker(
        location=(row['lat'], row['lon']),
        radius=5 + (row['total_taxi_activity'] / 2000),  # Size by total activity
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        tooltip=(f"Zone: {row['zone']}<br>Borough: {row['borough']}<br>Total Taxi Activity: {row['total_taxi_activity']}")
    ).add_to(taxi_cluster)

# Add a separate marker cluster for nearby CitiBike stations
bike_cluster = MarkerCluster(name="Nearby CitiBike Stations").add_to(nyc_map)
for _, row in nearby_stations.iterrows():
    # Add CitiBike stations as blue markers with tooltip for name and capacity
    folium.Marker(
        location=(row['lat'], row['lon']),
        icon=folium.Icon(color='blue', icon='bicycle'),
        tooltip=(f"Station: {row['name']}<br>Capacity: {row['capacity']}<br>Coordinates: ({row['lat']}, {row['lon']})")
    ).add_to(bike_cluster)

# Add layer control and display the map
folium.LayerControl().add_to(nyc_map)
nyc_map


Map Centering and Zoom:

The map is centered on New York City with a moderate zoom level, allowing you to view a broad area of NYC.
High-Density Taxi Zones:

Red circle markers represent high-density taxi zones (zones with high activity in taxi pick-ups and drop-offs).
The size of each red circle marker reflects the level of taxi activity in that zone, calculated using the total_taxi_activity value.
Each high-density zone marker includes a tooltip that shows:
The zone's name.
The borough in which the zone is located.
The total taxi activity (pickups and drop-offs).
Nearby CitiBike Stations:

Blue markers represent CitiBike stations located near these high-density taxi zones.
Each CitiBike station marker has an icon of a bicycle and includes a tooltip that shows:
The station’s name.
The station’s capacity (i.e., how many bikes it can hold).
The station’s geographic coordinates.
Marker Clustering:

Both the high-density taxi zones and CitiBike stations are grouped into clusters. This clustering keeps the map organized by grouping markers together at larger zoom levels. As you zoom in, the clusters will break down to show individual markers.
Layer Control:

You can toggle visibility for the "High-Density Taxi Zones" and "Nearby CitiBike Stations" layers using the layer control on the map

In [48]:
nyc_map.save("/Users/disastershubz/Documents/PROJECT/nyc_map.html")
